In [ ]:
!pip install ktrain

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
import ktrain
from ktrain import text

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/')

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
random_seed = 1234
x_train, x_test, y_train, y_test = train_test_split(train_df['review'], 
                                                    train_df['sentiment'], 
                                                    shuffle=True, 
                                                    test_size = 0.2, 
                                                    random_state=random_seed, 
                                                    stratify=train_df['sentiment'])

In [ ]:
# text.print_text_classifiers()
# siebert/sentiment-roberta-large-english
# distilbert-uncased-based 0.91 8e-5
# distilbert-base-uncased-finetuned-sst-2-english 1e-4 0.89
# distilbert-base-uncased-finetuned-sst-2-english 2e-5 0.941
# distilbert-base-uncased-finetuned-sst-2-english 2.5e-5 0.942
# distilbert-base-uncased-finetuned-sst-2-english 3e-5 0.922
# distilbert-base-uncased-finetuned-sst-2-english 5e-5 0.939
# distilbert-base-uncased-finetuned-sst-2-english 8e-5 0.922
# distilbert-base-uncased-finetuned-sst-2-english 6e-5 0.93

In [ ]:
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
t = text.Transformer(MODEL_NAME, maxlen=512, class_names=['0','1'])
trn = t.preprocess_train(np.array(x_train), np.array(y_train))
val = t.preprocess_test(np.array(x_test), np.array(y_test))
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=16)


preprocessing train...
language: en
train sequence lengths:
	mean : 197
	95percentile : 549
	99percentile : 858


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 206
	95percentile : 588
	99percentile : 889


In [ ]:
# learner.lr_find()

In [ ]:
#learner.lr_plot()

In [ ]:
learner.fit_onecycle(4e-5, 4)



begin training using onecycle policy with max lr of 4e-05...
Epoch 1/4
 108/1467 [=>............................] - ETA: 18:39:33 - loss: 0.2752 - accuracy: 0.9120

In [ ]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      2878
           1       0.94      0.94      0.94      2991

    accuracy                           0.94      5869
   macro avg       0.94      0.94      0.94      5869
weighted avg       0.94      0.94      0.94      5869



array([[2688,  190],
       [ 166, 2825]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)


In [ ]:
predictor.predict('Run tesseract to terrebly bad box file to make training data set.')

'0'

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/test.csv', index_col='ID')
df_test.head()

,review
ID,
22622,Robert Lansing plays a scientist experimenting...
10162,"Well I've enjoy this movie, even though someti..."
17468,First things first - though I believe Joel Sch...
42579,I watched this movie on the grounds that Amber...
701,A certain sexiness underlines even the dullest...


In [ ]:
 
test_df_sentiment = predictor.predict(df_test.review.to_list())
 
 
submission = pd.DataFrame(
    {'ID':df_test.index, 
     'sentiment': test_df_sentiment}
)
submission.head()
submission.to_csv('submission.csv', index=None)